In [5]:
import random

In [6]:
class TRIEST_BASE:
    def __init__(self, M = 100):
        self.M = M
        self.S = set()
        self.global_counter = 0
        self.local_counter = {}
        self.t = 0

    def sample_edge(self, Edge, t):
        if t <= self.M:
            return True
        if random.random() <= (self.M/t):
            random_edge = random.sample(self.S, 1)[0]
            self.S.remove(random_edge)
            self.update_counters('-', Edge)
            return True
        return False

    def update_counters(self, operation, edge):
        # edge E = (u,v)
        u = edge[0]
        v = edge[1]
        neighborhood_of_u = set()
        neighborhood_of_v = set()
        # construct neighborhood of u
        for one_edge in self.S:
            if u == one_edge[0]:
                neighborhood_of_u.add(one_edge[1])
            if u == one_edge[1]:
                neighborhood_of_u.add(one_edge[0])
            # construct neighborhood of v
            if v == one_edge[0]:
                neighborhood_of_v.add(one_edge[1])
            if v == one_edge[1]:
                neighborhood_of_v.add(one_edge[0])
        # shared neighborhood of u and v
        shared_neighborhood = set.intersection(neighborhood_of_u, neighborhood_of_v)
        # update counters
        for c in shared_neighborhood:
            if operation == '+':
                self.global_counter += 1
                self.local_counter[c] = self.local_counter.get(c, 0) + 1
                self.local_counter[u] = self.local_counter.get(u, 0) + 1
                self.local_counter[v] = self.local_counter.get(v, 0) + 1

            if operation == '-':
                self.global_counter -= 1
                self.local_counter[c] = self.local_counter.get(c, 0) - 1
                if self.local_counter[c] <= 0:
                    del self.local_counter[c]
                self.local_counter[u] = self.local_counter.get(u, 0) - 1
                if self.local_counter[u] <= 0:
                    del self.local_counter[u]
                self.local_counter[v] = self.local_counter.get(v, 0) - 1
                if self.local_counter[v] <= 0:
                    del self.local_counter[v]

    def run_triest_base(self, streams):

        for element in streams:
            self.t += 1
            if self.sample_edge(element, self.t):
                self.S.add(element)
                self.update_counters('+', element)

        eps = (self.t * (self.t - 1) * (self.t - 2)) / (self.M * (self.M - 1) * (self.M - 2))

        eps = max(1, eps)
        print('Epsilon is ', eps)
        # estimation for the global triangle count
        est_gc = eps * self.global_counter
        return est_gc

In [7]:
def undirected_edge(u,v):
    if u < v:
        return (u,v)
    else:
        return (v,u)

# import dataset
streams = set()

with open("CA-HepTh.txt") as f:
    for line in f:
        if line[0] == '#':
            continue
        edge = line.split()
        if edge[0] != edge[1]:
            streams.add(undirected_edge(edge[0], edge[1]))
        # if size_stream == 10000:
        #     break

print('The amount of edges of Data Stream contains', len(streams))


The amount of edges of Data Stream contains 25973


In [8]:
triest_base = TRIEST_BASE(8000)
print('the value of M is', triest_base.M)
glo_tri_counter = triest_base.run_triest_base(streams)
print('Estimation for the global triangle count is', glo_tri_counter)

triest_base = TRIEST_BASE(len(streams))
print('the value of M is', len(streams))
true_glo_tri_counter = triest_base.run_triest_base(streams)
print('Actual global triangle count is', true_glo_tri_counter)

error_count = abs(true_glo_tri_counter - glo_tri_counter)
print('Error triangles are', error_count)
error_rate = error_count/ true_glo_tri_counter
print('Error rate is', error_rate)



the value of M is 8000
Epsilon is  34.230173339207724
Estimation for the global triangle count is 25569.93948438817
the value of M is 25973
Epsilon is  1
Actual global triangle count is 28339
Error triangles are 2769.060515611829
Error rate is 0.0977120052087875
